In [ ]:
#This performs MH algorithm on a partially-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import norm

In [ ]:
#initializations
A_true=5      #indicate true value of A
B_true=0.2    #indicate true value of B
N=5000        #length of the chain
A=np.zeros(N+1)
B=np.zeros(N+1)
A[0]=1        #initialize the Markov Chain A
B[0]=1        #initialize the Markov Chain B

var_A=0.2       #set proposal variance for A
var_B=0.02    #set proposal variance for B

Ashape=4    #set the shape of the Gamma prior for A
Ascale=1      #set the scale of the Gamma prior for A
Bshape=0.1   #set the shape of the Gamma prior for B
Bscale=1      #set the scale of the Gamma prior for B
T=50          #end time for observations

In [ ]:
#import saved data
full_x=pd.read_csv("https://raw.githubusercontent.com/mvcortez/IWOMB_2024_Inference/main/full.csv").values
t_x=pd.read_csv("https://raw.githubusercontent.com/mvcortez/IWOMB_2024_Inference/main/full_time.csv").values

#subsample the trajectory on unit intervals
t = np.arange(T+1)
x=np.zeros(len(t))
for i in range(len(t)):
    idx = np.max(np.where(t_x<=t[i]))
    x[i]=full_x[idx]

#Plot the subsampled trajectory
plt.scatter(np.arange(len(t)),x,color='r')
plt.plot(np.arange(len(t)),x,color='r')
plt.title('Partially observed Birth-death process')
plt.xlabel('time')
plt.ylabel('Molecular Count')
plt.show()

In [ ]:
def init_reaction(x,s):
    q=np.zeros([2,len(x)-1])
    for i in range(s-1):
        if x[i+1]-x[i]>0:
            q[0,i]=np.floor((4/3)*(x[i+1]-x[i]))
            q[1,i]=np.floor((1/3)*(x[i+1]-x[i]))
        else:
            q[0,i]=-np.floor((1/3)*(x[i+1]-x[i]))
            q[1,i]=-np.floor((4/3)*(x[i+1]-x[i]))
    r=q.astype(int)        
    return r  

def hom_poisson_process(r,x,A,B,size,r_dim):
    t_birth=np.zeros([r_dim[0],size-1])
    t_death=np.zeros([r_dim[1],size-1])
    for j in range(size-1):
        if r[0,j]>0:
            t_birth[:r[0,j],j]=j+np.sort(np.random.uniform(0,1,r[0,j]))
        h_k1=B*x[j]
        h_k2=B*x[j+1]
        if r[1,j]>0:       
            t_death[:r[1,j],j]=np.sort(np.random.uniform(0,1,r[1,j]))
            if x[j]!=x[j+1]:
                for p in range(r[1,j]):
                    t_death[p,j]=j+((np.sqrt(h_k1**2+(h_k2**2-h_k1**2)*t_death[p,j])-h_k1)/(h_k2-h_k1))
            else:
                t_death[:,j]+=j
    return [t_birth, t_death]

def loglikelihood_x(z,r,t_b,t_d,A,B,size):
    size1=len(z)
    like=np.zeros(size-1)
    integral=np.zeros((size-1))
    for j in range(size-1):
        a=np.zeros([r[0,j]])
        b=np.zeros([r[1,j]])
        w=np.zeros([r[0,j]+r[1,j]])
        y=np.zeros([2+r[0,j]+r[1,j]])
        h0_vector=None
        y[0]=z[j]
        y[len(y)-1]=z[j+1]
        for i in range(r[0,j]):
            a[i]=t_b[i,j]
        for i in range(r[1,j]):
            b[i]=t_d[i,j]
        m=np.sort(np.ndarray.flatten(np.append(a,b, axis=0)))
        for k in range(len(m)):
            if np.any(a==m[k])==True:
                y[k+1]=y[k]+1
                w[k]=np.log(A)
            else:
                y[k+1]=y[k]-1
                w[k]=np.log(B*y[k-1])
        #y=np.ndarray.flatten(y)        
        like[j]=np.sum(w)
        h0_vector=A+B*y    
    loglik=np.sum(like)-np.trapz(A+B*np.ndarray.flatten(z),np.arange(len(z)))
    return loglik

def accept_rate_react(x,r,prop,i,A,B):
    prop_like=poisson.pmf(prop[0],A)*poisson.pmf(prop[1],0.5*B*(x[i]+x[i+1]))
    current_like=poisson.pmf(r[0],A)*poisson.pmf(r[1],0.5*B*(x[i]+x[i+1]))  
    rate=np.minimum(1,prop_like/current_like)
    return rate
 
def accept_rate_x(var,x,r,t_b,t_d,A,B,size,prop,shape,scale1):
    if var=='birth':
        prop_like=loglikelihood_x(x,r,t_b,t_d,prop,B,size)+np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood_x(x,r,t_b,t_d,A,B,size)+np.log(gamma.pdf(A,a=shape,scale=scale1))
    else:
        prop_like=loglikelihood_x (x,r,t_b,t_d,A,prop,size) +np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood_x(x,r,t_b,t_d,A,B,size) +np.log(gamma.pdf(B,a=shape,scale=scale1))
    rate=np.minimum(1,np.exp(prop_like-current_like))
    return rate

In [ ]:
#Initialize reaction numbers and times
r=init_reaction(x,len(x))

#2-block MH algorithm
accept=0
for i in range(N):
    #Perform approximate block updating
    for j in range(len(x)-1):   
        r_vprop=[-1,-1]
        while r_vprop[0]<0 or r_vprop[1]<0: 
            jump=np.random.poisson(2)-np.random.poisson(2)
            r_prop=r[0,j]+jump
            r_vprop=[r_prop, r_prop-(x[j+1]-x[j])]
        rate=accept_rate_react(x,r[:,j],r_vprop,j,A[i],B[i])
        if np.random.uniform(0,1,1)<rate:
            r[:,j]=r_vprop      
    r_dim=np.max(r,axis=1).astype(int)        
    [t_birth,t_death]=hom_poisson_process(r,x,A[i],B[i],len(x),r_dim)     
    
    #Perform Gibbs sampling
    if i % 2 != 0:     #MH for A
        propA=-1
        while propA<0:
            propA=A[i]+np.random.normal(0,var_A,1)
        rate=accept_rate_x('birth',x,r,t_birth,t_death,A[i],B[i],len(x),propA,Ashape,Ascale)
        if np.random.uniform(0,1,1)<rate:
            accept+=1
            A[i+1]=propA
            B[i+1]=B[i]
        else:
            A[i+1]=A[i]
            B[i+1]=B[i]
    else:              #MH for B
        propB=-1
        while propB<0:
            propB=B[i]+np.random.normal(0,var_B,1)
        rate=accept_rate_x('death',x,r,t_birth,t_death,A[i],B[i],len(x),propB,Bshape,Bscale)
        if np.random.uniform(0,1,1)<rate:
            accept+=1
            B[i+1]=propB
            A[i+1]=A[i]
        else:
            B[i+1]=B[i]
            A[i+1]=A[i]
    
    if (i+1)%100 == 0:
        print('Iteration %d' %(i+1),'A = %1.3f' %(A[i+1]), 'B = %1.3f' %(B[i+1]) )
        
acceptance_rate=accept/N            

In [ ]:
#plot the trajectory and autocorrelation function
t0=0     #indicate the starting time step 
tf=N     #indicate the terminal time step 
nA=A[t0:tf]
nB=B[t0:tf]
plt.plot(nA, nB, '.r-')
plt.title('Sample Trajectory, Acceptance Rate %1.3f' %acceptance_rate)
plt.xlabel('A')
plt.ylabel('B')
plt.show()

In [ ]:
#Burn-in and thinning
burn_in = 1000
thin_factor=5
bA=A[burn_in:]
bB=B[burn_in:]
tA=np.zeros(np.floor((N-burn_in)/thin_factor).astype(int))
tB=np.zeros(np.floor((N-burn_in)/thin_factor).astype(int))
count=0
for i in range(N-burn_in):
    if (i+1)%thin_factor==0:
        tA[count]=bA[i]           #thinned chain A
        tB[count]=bB[i]           #thinned chain B
        count+=1
        
print('The thinned A samples have mean %1.3f and variance %1.3f.' %(np.mean(tA), np.var(tA)))        
print('The thinned B samples have mean %1.3f and variance %1.3f.' %(np.mean(tB), np.var(tB)))

In [ ]:
#plot autocorrelation function of the thinned samples
plot_acf(tA,lags=len(tA)-1)
plt.title('Autocorrelation function of the thinned A samples')
plt.show()

plot_acf(tB,lags=len(tB)-1)
plt.title('Autocorrelation function of the thinned B samples')
plt.show()

In [ ]:
#Histogram of thinned samples
plt.hist(tA, density=True,bins=20)
plt.title('Histogram of A Samples')
plt.axvline(x=A_true,color='r', linestyle='--', label='True Value = %1.3f' %A_true)
plt.axvline(x=np.mean(tA),color='y', linestyle='--', label='Sample Mean = %1.3f' %np.mean(tA))
plt.legend()
plt.xlabel('A')
plt.ylabel('Normalized Count')
plt.show()

plt.hist(tB, density=True,bins=20)
plt.axvline(x=B_true,color='r', linestyle='--', label='True Value = %1.3f' % B_true)
plt.axvline(x=np.mean(tB),color='y', linestyle='--', label='Sample Mean =%1.3f' % np.mean(tB))
plt.legend()
plt.title('Histogram of B Samples')
plt.xlabel('B')
plt.ylabel('Normalized Count')
plt.show()